# Simple Chat using OCI Generative AI

The notebook sets up a simple chat application using Oracle Cloud Infrastructure (OCI) Generative AI services. Here's an overview of the entire notebook:

1. Imports and Environment Setup:

    - Imports necessary libraries such as os, ads, oracledb, dotenv, and oci_genai.
    - Loads environment variables from a .env file using load_dotenv().

2. Embeddings Initialization:

    - Creates an instance of OCIGenAIEmbeddings using environment variables for model ID, service endpoint, and compartment ID.

3. Database Connection:

    - Establishes a connection to an Oracle database using oracledb.connect() with credentials retrieved from environment variables.

4. Vector Store Setup:

    - Defines the table name "MY_DOCS" as knowledge base.
    - Initializes the OracleVS vector store with the database connection, embeddings, and table name.

1. Retriever Setup:

    - Converts the vector store into a retriever using vector_store.as_retriever().

2. Prompt Template Definition:

    - Defines a prompt template for the RAG model to use when generating answers.
    - The template instructs the assistant to use retrieved context fragments to answer questions.

3. Chat Model Initialization:

    - Creates an instance of ChatOCIGenAI with specified parameters, including model ID, service endpoint, compartment ID, and other model settings.

4. RAG Model Setup:

    - Combines the chat model and retriever into a RetrievalQA model using RetrievalQA.from_chain_type().
    - Uses the defined prompt template for generating answers.

5. Example Usage:

    - Demonstrates how to use the RAG model to generate responses to user inputs.
    - Sends a prompt to the model and prints the generated response.

In [1]:
import os
import oracledb
from dotenv import load_dotenv
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import OracleVS
import oracledb

load_dotenv()
%load_ext autoreload
%autoreload 2
    


In [3]:

default_path=""
AUTH_TYPE = "API_KEY"
CONFIG_PROFILE = "DEFAULT"
embeddings = OCIGenAIEmbeddings(
            model_id=os.getenv('CON_GEN_AI_EMB_MODEL_ID'),
            service_endpoint=os.getenv('CON_GEN_AI_SERVICE_ENDPOINT'),
            compartment_id=os.getenv('CON_GEN_AI_COMPARTMENT_ID'),
            truncate="NONE",
            auth_file_location=default_path+"oci/config",
            auth_type=AUTH_TYPE,
            auth_profile=CONFIG_PROFILE
        )


default_path = ""
connection = oracledb.connect(
    user=os.getenv('CON_ADB_DEV_USER_NAME'), 
    password=os.getenv('CON_ADB_DEV_PASSWORD'), 
    dsn=os.getenv('CON_ADB_DEV_SERVICE_NAME'),
    config_dir=default_path+"oci",
    wallet_location=default_path+"oci",
    wallet_password=os.getenv('DB_WALLET_PASSWORD')
    )


#vector store table name
table_name = "MY_DOCS"

vector_store = OracleVS(connection, embeddings, table_name)


retriever = vector_store.as_retriever(
    # search_kwargs={ 'k': 100} <- parameter to define the number of documents to retrieve
)

rag_prompt_template = """You are an assistant for question-answering tasks.
            Please use only the following retrieved context fragments to answer the question.
            If you don't know the answer, say you the cuestion is out of my context.
            Always use all available data.:
            {context}
            Question: {question}
            """

rag_prompt = PromptTemplate.from_template(rag_prompt_template)


llm = ChatOCIGenAI(
        model_id         = os.getenv("CON_GEN_AI_CHAT_MODEL_ID"),
        service_endpoint = os.getenv("CON_GEN_AI_SERVICE_ENDPOINT"),
        compartment_id   = os.getenv("CON_GEN_AI_COMPARTMENT_ID"),
        provider         = "meta",
        is_stream        = True,
        auth_type        = os.getenv("CON_GEN_AI_AUTH_TYPE"),
        model_kwargs     = {
            "max_tokens"        : 1024,
            "temperature"       : 0.6,
            "top_p"             : 0.7,
            "top_k"             : 20,
            "frequency_penalty" : 0
        }
    )

rag = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt}
)


In [4]:
print(rag.invoke("what is Oracle AI Vector Search?"))

{'query': 'what is Oracle AI Vector Search?', 'result': 'Oracle AI Vector Search is designed for Artificial Intelligence (AI) workloads and allows you to query data based on semantics, rather than keywords. It stores vector embeddings, which are mathematical vector representations of data points, and these vector embeddings describe the semantic meaning behind content such as words, documents, audio tracks, or images.'}


In [5]:
print(rag.invoke("Can you talk about peruvian food?"))

{'query': 'Can you talk about peruvian food?', 'result': 'The question is out of my context.'}
